In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U langchain-community
!pip install bitsandbytes
!pip install -U bitsandbytes transformers
!pip install faiss-gpu-cu12
!pip install -U langchain-community
!pip install datasets
!pip install rank_bm25
!pip install trl
!pip install kiwipiepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [3]:
import os
from peft import PeftConfig
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [4]:
import pandas as pd
import numpy as np
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [5]:
data_path = "/content/drive/MyDrive/"

In [6]:
#불러오기
FINETUNED_MODEL = "/content/drive/MyDrive/qloraV2"

peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [7]:
train = pd.read_csv(data_path+"train2.csv" )
test = pd.read_csv(data_path+"test2.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [8]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [9]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"'{row['공사종류(중분류)']}' 공사 중 {row['공종(중분류)']}' 작업에서  '{row['작업프로세스']} 작업프로세스 진행중 {row['인적사고']} 발생 했습니다 "
            f"사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "templeate" : row['templeate'],
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [10]:
# 훈련 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"'{row['공사종류(중분류)']}' 공사 중 {row['공종(중분류)']}' 작업에서  '{row['작업프로세스']} 작업프로세스 진행중 {row['인적사고']} 발생 했습니다 "
            f"사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "templeate" : row['templeate'],
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [11]:
# 학습 모델용
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # 4비트 로드 활성화
    bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
    bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
)

In [12]:
# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [13]:
# QLoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)
# QLoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
############### 조심 ###################

In [ ]:
# # 일반 모델용
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )

In [ ]:
# model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=bnb_config,
#     torch_dtype=torch.float16,
#     trust_remote_code=True,
#     device_map="auto"
# )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [14]:
# bm25 토크나이저로 한국어 토크나이저 사용 하기 위해서 불러옴
from kiwipiepy import Kiwi
kiwi = Kiwi()

def ko_kiwi_tokenizer(text: str):
    # Kiwi 토크나이저는 각 토큰에 대한 다양한 정보를 반환합니다.
    # 여기서는 토큰의 표면 형태(텍스트)만 추출합니다.
    tokens = kiwi.tokenize(text)
    return [token[0] for token in tokens]

In [15]:
import torch
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document

# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]
docs = [Document(page_content=doc) for doc in train_documents]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)


# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
bm25_retriever = BM25Retriever.from_documents(docs, tokenizer=ko_kiwi_tokenizer , k =3 )

# 앙상블 리트리버 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0.5]  # 각 리트리버에 동일 가중치 부여 (가중치 합은 1.0)
)




<ipython-input-15-17b25d2e2ef2>:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample = False,
    #temperature=0.1, # 0~1 까지 0에 가까울 수록 보수적인 답변 즉 창의성이 떨어짐
    return_full_text = False,
    max_new_tokens=128, # 문장 최대 길이 조정
    batch_size=16  # 배치 크기 지정
)

Device set to use cuda:0


In [42]:
#- 답변은 한 문장으로 작성하며, 15~30단어 이내로 구성합니다.
# 1. 프롬프트 템플릿 정의 (필요한 입력 변수: templeate, context, question)
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
- 질문에 대해 재발 방지 대책 및 향후 조치 계획을 한문장으로 답변하세요.
- 재발 방지 대책 및 향후 조치 계획이 2개이상 이면 ,을 사용해서 한문장으로 나열하세요
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 주어나 부연 설명, 연결어는 제거하고 핵심 용어만 사용합니다.
- 반드시 실행 명령문 형식으로 작성하고, 구체적인 조치(예: 안전교육, 재교육, 정기점검 등)를 포함해야 합니다.
- 특수문자, 리스트, 목차 등은 사용하지 않습니다.
- 제공된 가이드라인 기반으로 작성하고 참고 문서와 가이드라인에서 사용되는 단어를 활용해서 답변하세요.

### 가이드라인:
{templeate}

### 참고 문서:
{context}

### 질문:
{question}

### 답변:
"""

# 2. PromptTemplate 생성 (input_variables에 "templeate", "context", "question" 모두 포함)
prompt = PromptTemplate(
    input_variables=["templeate", "context", "question"],
    template=prompt_template,
)


In [43]:
from langchain.chains import LLMChain
def retrieve_context(question, k=3):
    docs = ensemble_retriever.get_relevant_documents(question)
    # 문서를 하나의 문자열로 합치거나 원하는 방식으로 포맷팅
    context = "\n".join([doc.page_content for doc in docs])
    return context

def generate_answer(question, templeate):
    context = retrieve_context(question)
    inputs = {
        "templeate": templeate,
        "context": context,
        "question": question,
    }
    answer = llm_chain.predict(**inputs)
    return answer
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
# LLMChain 정의 (prompt는 기존에 사용하던 템플릿 활용)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [44]:
tokenizer.padding_side = "left"

In [45]:
from torch.utils.data import DataLoader
from datasets import Dataset
import torch
from tqdm import tqdm
import psutil

# Matmul 성능 최적화
torch.set_float32_matmul_precision('high')

# 테스트 데이터 준비
test_questions = combined_test_data['question'].tolist()
test_templeates = combined_test_data['templeate'].tolist()
test_dataset = Dataset.from_dict({
    "question": test_questions,
    "templeate": test_templeates
})

# DataLoader 설정
batch_size = 16
num_workers = 0  # GPU 환경에서는 num_workers=0이 적절함
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

test_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

def process_batch(batch):
    batch_outputs = []
    for question, templeate in zip(batch['question'], batch['templeate']):
        answer = generate_answer(question, templeate)
        batch_outputs.append(answer)
    return batch_outputs

test_results = []
for batch in tqdm(test_dataloader, desc="처리 진행률"):
    batch_results = process_batch(batch)
    test_results.extend(batch_results)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))


테스트 실행 시작... 총 테스트 샘플 수: 964


처리 진행률: 100%|██████████| 61/61 [27:15<00:00, 26.80s/it]


테스트 실행 완료! 총 결과 수: 964


In [ ]:
# from accelerate import Accelerator
# from torch.utils.data import DataLoader
# from datasets import Dataset
# from tqdm import tqdm

# # Accelerator 객체 생성 (단일 GPU 환경)
# accelerator = Accelerator()

# # 테스트 데이터 준비
# test_questions = combined_test_data['question'].tolist()
# test_dataset = Dataset.from_dict({"question": test_questions})

# # DataLoader 설정 (batch_size, num_workers 등은 환경에 맞게 조정)
# batch_size = 16
# num_workers = 0  # GPU 환경에서는 일반적으로 num_workers=0 사용
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# # Accelerator를 통해 DataLoader 준비 (GPU로 데이터를 전달)
# test_dataloader = accelerator.prepare(test_dataloader)

# test_results = []
# print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

# # 배치 처리 함수
# def process_batch(batch):
#     # 각 배치의 "question"에 대해 입력 형식으로 변환
#     batch_inputs = [{"query": q} for q in batch["question"]]
#     batch_outputs = qa_chain.batch(batch_inputs)
#     return batch_outputs

# # DataLoader를 이용한 배치 처리
# for batch in tqdm(test_dataloader, desc="처리 진행률"):
#     batch_results = process_batch(batch)
#     # 단일 GPU 환경이므로 gather() 호출은 생략해도 무방합니다.
#     test_results.extend(batch_results)

# print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

Parameter 'function'=<function process_batch at 0x7b42307b25c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


테스트 실행 시작... 총 테스트 샘플 수: 964


Map:   0%|          | 0/964 [00:00<?, ? examples/s]

ArrowInvalid: Could not convert Document(metadata={}, page_content="Q: '건축물' 공사 중 철근콘크리트공사' 작업에서  '타설작업 작업프로세스 진행중 부딪힘 발생 했습니다 사고 원인은 '버림 콘크리트 하부 지반침하로 인하여 콘크리트 펌프카 아웃트리거 침하'입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?\nA: 해당 구간의 버림 콘크리트 제거 및 지반 치환 후 버림 콘크리트 타설 예정.") with type Document: did not recognize Python value type when inferring an Arrow data type

In [48]:
test_results

['펌프카 아웃트리거 지반 상태 정기 점검 및 보강 강화, 아웃트리거 펼친 길이 일치 확인 및 조정 장치 설치, 작업 위치 최적화 및 무게중심 분산 유도, 작업 전 TBM 실시 및 위험요인 공유, 재발 방지 교육 실시 및 안전 점검 체크리스트 도입.',
 '작업자 안전 교육 강화 및 보안면 착용 의무화, 숫돌 사용 교육 실시.',
 '작업자 안전의식 강화 및 계단 주변 안전표지 설치와 함께 작업 전 안전 교육 실시.',
 '작업 발판 주변 벽돌 잔재 철저히 정리정돈하고, 안전 교육 강화를 통한 향후 조치 계획.',
 '작업자 안전 교육 강화 및 현장 내 안전난간 설치 의무화를 통한 재발 방지 대책 및 향후 조치 계획 실시.',
 '안전발판 설치 철저와 작업절차 준수 교육 실시 계획.',
 '작업자 불안전 행동 방지를 위한 안전교육 강화 및 매설작업 시 안전고리 이중결속 준수 의무화.',
 '작업자 안전모 착용 의무화 및 TSC GIRDER 조립 시 안전거리 확보 교육 실시를 통한 재발 방지 대책 및 향후 조치 계획.',
 '이동식틀비계 작업발판 사용 시 안전발판 설치 및 정기적인 안전점검 실시.',
 '작업자 대상 철골 해체작업 안전 교육 강화 및 현장 위험요소 제거를 위한 정기적인 안전 점검 실시.',
 '작업 전 고령 작업자 건강상태 점검 의무화 및 현장 내 안전시설물 설치 강화. 고령 작업자에 대한 정기적인 건강검진 실시와 작업 전 안전 확인 절차 의무화.',
 '장비 안전 점검 강화 및 철판 궤도 설치 기준 엄격히 준수하도록 현장 안전 교육 실시.',
 '작업 전 안전 교육 강화 및 작업 중 관리자의 현장 상주 및 실시간 작업 모니터링 실시.',
 '자재 관리 직원의 안전 교육 강화 및 작업 중 자재 이동 시 안전 통제 철저 시행.',
 '작업자 안전교육 강화 및 작업환경 정기 점검 실시.',
 '지하작업 시 합판 고정 강화 및 작업자 안전 교육 실시.',
 '작업자 안전 확보를 위한 특별안전교육 실시 및 고정핀 제거 시 거푸집 안정성 확인 절차 의무화

''

In [ ]:

import pickle

# test_results를 pickle 파일로 저장
with open("test_results.pkl", "wb") as f:
    pickle.dump(test_results, f)

print("test_results가 'test_results.pkl' 파일에 저장되었습니다.")


test_results가 'test_results.pkl' 파일에 저장되었습니다.


In [ ]:
# 'result' 값들만 추출
results = [item['result'] for item in test_results]

# 추출한 결과를 이용해 임베딩 생성
pred_embeddings = embedding.encode(results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

# 최종 결과 저장 (submission의 첫 번째 열에 'result' 값 할당)
submission.iloc[:, 1] = results
submission.iloc[:, 2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('submission-EXAONE-3.5-32B-Instruct.csv', index=False, encoding='utf-8-sig')


(964, 768)


In [49]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

(964, 768)


In [53]:
submission = pd.read_csv(data_path+"sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()
# 최종 결과를 CSV로 저장
submission.to_csv('템플릿적용V2.csv', index=False, encoding='utf-8-sig')